# Goal: Retrieve tweets from the tweet ids store in a CSV file
*** 

References:
1. https://morioh.com/p/eabbdfdd1f0b
2. https://www.kaggle.com/anshuls235/data-wrangling-explained



## Step 1: Get twitter handle by reading credentials store in a json file

In [6]:
import json
import csv
import time
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream #TwitterStreamManager
from tweepy import API

import os
os.chdir("c://ConfidentialFolder/")
# retrieve credentials keys from the json file
my_tweet_cred_file = 'twitter-credentials.json'
json_file = open(my_tweet_cred_file)
mytweetcred = json.load(json_file)
json_file.close() 

# store credentials in the local variables 
consumer_key = mytweetcred['consumer_key']
consumer_key_secret = mytweetcred['consumer_secret']
access_token = mytweetcred['access_token']
access_token_secret = mytweetcred['access_token_secret']

# pass the credentials to the authentication process and get API handle
auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

## Step 2: Declare file names that need to retrieve

In [16]:
# declare folder path
file_path = "C://SHU/ResearchAndDissertation/final-dissertation/CoAID-master/"

# declare Claims (fake and real, tweets and replies to tweets)
fake_claim_tweets = "ClaimFakeCOVID-19_tweets"
fake_claim_tweets_replies = "ClaimFakeCOVID-19_tweets_replies"
real_claim_tweets = "ClaimRealCOVID-19_tweets"
real_claim_tweets_replies = "ClaimRealCOVID-19_tweets_replies"

# declare News (fake and real, tweets and replies to tweets)
fake_news_tweets = "NewsFakeCOVID-19_tweets"
fake_news_tweets_replies = "NewsFakeCOVID-19_tweets_replies"
real_news_tweets = "NewsRealCOVID-19_tweets"
real_news_tweets_replies = "NewsRealCOVID-19_tweets_replies"

In [1]:
# from json.encoder import JSONEncoder
tweet = api.get_status("1242474839966765056")
json_str = json.dumps(tweet._json)
# JSONEncoder().encode(tweet)
# datajson = json.dumps(tweet.__dict__)
# datajson = json.dumps(tweet)
print(json_str)

## Step3: Read CSV file

In [2]:
inputfile = file_path+fake_claim_tweets+".csv"
outputfile = file_path+fake_claim_tweets+"_tweets.csv"

def ReadTweetTextFromCSV(inputfile):
    # fill tweetid_list with data
    tweetid_list = []
    with open(inputfile, 'r') as csvfile:
        line_reader = csv.reader(csvfile, delimiter=',', quotechar="\"")
        for row in line_reader:
            tweetid_list.append({'index':row[0], 'tweet_id':row[1]})

    # once tweetid_list got filled with data, loop through it
    sleep_time = 2
    fetched_tweet_ds = []
    for tweet in tweetid_list: 
        try:
            fetched_tweet = api.get_status(tweet["tweet_id"])
            print("Tweet fetched" +fetched_tweet.text)
            tweet["text"] = fetched_tweet.text
            fetched_tweet_ds.append(tweet)
            time.sleep(sleep_time)
        except: 
            print("Exception occured - no:2")
            continue
    return fetched_tweet_ds

def WriteTweetTextToCSV(inputfile, outputfile):
    # fill tweetid_list with data
    tweetid_list = []
    with open(inputfile, 'r') as csvfile:
        line_reader = csv.reader(csvfile, delimiter=',', quotechar="\"")
        for row in line_reader:
            tweetid_list.append({'index':row[0], 'tweet_id':row[1]})

    # once tweetid_list got filled with data, loop through it
    sleep_time = 2
    fetched_tweet_ds = []
    for tweet in tweetid_list: 
        try:
            fetched_tweet = api.get_status(tweet["tweet_id"])
            print("Tweet fetched" +fetched_tweet.text)
            tweet["text"] = fetched_tweet.text
            fetched_tweet_ds.append(tweet)
            time.sleep(sleep_time)
        except: 
            print("Exception occured - no:2")
            continue

    # now open outputfile in a write mode and write the retrieved tweets back to it
    with open(outputfile, 'w') as csvfile:
        line_writer = csv.writer(csvfile, delimiter=',', quotechar="\"")
        for tweet in fetched_tweet_ds:
            try:
                line_writer.writerow([tweet["tweet_id"], tweet["text"]])
            except Exception as e:
                print(e)           

In [56]:
import pandas as pd 
df_twitter_archive = pd.read_csv(file_path+fake_claim_tweets_replies+".csv")
for tweet_id in df_twitter_archive.tweet_id:
    try:
        tweet_json = api.get_status(tweet_id, tweet_mode = 'extented')
        with open('tweet_json.txt', mode='a') as file:
#             json.dump(tweet_json, file)
            file.write(json.dumps(tweet_json._json))
            file.write('\n')
    except Exception as e:
        print(str(tweet_id) + ': ' + str(e))

1248235247809867777: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
1248235247809867777: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
1248235247809867777: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
1248235247809867777: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
1248235247809867777: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
1248235247809867777: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
1248235247809867777: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
1248235247809867777: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]
1242669446478671872: [{'code': 179, 'message': 'Sorry, you are not authorized to see this status.'}]
1242669446478671872: [{'code': 179, 'message': 'Sorry, you are not authorized to see this status.'}]
1240723147630018560: [{'code': 144, 'message': 'No status found with that ID.'}]
1237061437400305664: [{'code': 179, 'message': 'Sorry

In [57]:
lists = [] #an empyty list to store a dictionaries
with open('tweet_json.txt') as file:
    lines = file.read().splitlines()
    for line in lines:
        data = json.loads(line)
        row = {
            'tweet_id'      : data['id'],
            'retweet_count' : data['retweet_count'],
            'favorite_count': data['favorite_count']
        }
        lists.append(row)
df_tweet_data = pd.DataFrame(lists,columns=['tweet_id','retweet_count','favorite_count'])

In [71]:
df_tweet_data.tweet_id.nunique()

127

In [68]:
df_tweet_data.sort_values(by="retweet_count", ascending=False)

,tweet_id,retweet_count,favorite_count
22,1251668952993927170,95,176
14,1251668952993927170,95,176
24,1251668952993927170,95,176
23,1251668952993927170,95,176
21,1251668952993927170,95,176
...,...,...,...
318,1248621932104151042,0,0
317,1248621932104151042,0,0
316,1248621932104151042,0,0
315,1248621932104151042,0,0


In [64]:
# df_twitter_archive
df_twitter_archive.tweet_id.nunique()

131